In [ ]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [ ]:
import json
 
f = open('keys.json')

data = json.load(f)
clientID = data['ClientID6']
ClientSecret = data['ClientSecret6']


# authentication 
credentials = SpotifyClientCredentials(client_id=clientID, client_secret=ClientSecret)
sp = spotipy.Spotify(client_credentials_manager=credentials)



In [ ]:

def userToPlaylistList(username):
    userPlaylists = sp.user_playlists(username)["items"]
    userLinks = []
    for i in userPlaylists:
        userLinks.append(i["external_urls"]["spotify"])
    return userLinks
        

In [ ]:


# This function parses through a playlist and saves song data in csvs, with an offset optional


lis = []

def playlistData(url,offset=0):
    playlistUri = url.split("/")[-1].split("?")[0]
    limit = 100

    column_names = [
        'name',
        'artist',
        'url',
        'danceability',
        'energy',
        'key',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo'
    ]

    playlistDf = pd.DataFrame(columns=column_names)
    count = 4
    while True:
        # Fetching a batch of tracks from the playlist
        response = sp.playlist_tracks(playlistUri, offset=offset, limit=100, fields='items.track(name,artists,name,album,name,external_urls(spotify)),next')
        listOfTracks = response['items']
        lis = listOfTracks
        # Extract the track data and audio features
        for i, item in enumerate(listOfTracks):
            print(i)
            track = item['track']
            if 'spotify' in track['external_urls']:
                url = track['external_urls']['spotify']
                artistName = track['artists'][0]['name']
                trackName = track['name']
                try:
                    audioFeatures = sp.audio_features([url])[0]
                    time.sleep(1)
                    rowData = [trackName, artistName,url, audioFeatures['danceability'], audioFeatures['energy'],
                               audioFeatures['key'], audioFeatures['loudness'], audioFeatures['mode'], audioFeatures['speechiness'],
                               audioFeatures['acousticness'], audioFeatures['instrumentalness'], audioFeatures['liveness'],
                               audioFeatures['valence'], audioFeatures['tempo']]
                    playlistDf.loc[len(playlistDf)] = rowData
                except spotipy.SpotifyException as e:
                    if e.http_status == 429:
                        print(e.headers)
                    print("ooo")
        # each 100 requests, save as csv & reset dataframe
        print("./data/playlist----"+str(count)+".csv")
        playlistDf.to_csv("./data/playlist---"+str(count)+".csv")
        count+=1
        playlistDf = pd.DataFrame(columns=column_names)
        
        # Check if there are more tracks to fetch
        if response['next'] is not None:
            offset += limit
        else:
            break

    return playlistDf




# Parses through all of my friends' playlists
users = ['12176828559','bikramkohli007','crazydoc1','jamesl3248','eric_chungg','johnwarzecha-us','1270601874']
allPlaylists = []
for user in users:
    allPlaylists += userToPlaylistList(user)

    
playlist_df = playlistData(allPlaylists[56],300)
    
count = 57
for daPlaylist in allPlaylists[57::]:
    print("playlist "+str(count))
    playlist_df = playlistData(daPlaylist)
    count+=1

